In [20]:
# input_path = ["s3://web-parse-huawei/CC/pre-dedup/v002/unique_html/CC-MAIN-2024-33/"]
# input_df = read_any_path(spark, ",".join(input_path), config)
        
# s3://private-cooperate-data/zh-web-baijiahao/Articles/baijiahao_Articles_202401.jsonl
file_paths = ['s3://private-cooperate-data/zh-web-baijiahao/Articles/baijiahao_Articles_202401.jsonl']
    # 's3://data-warehouse/sufutao/llm_anno/content_only/textbook_content_20pages_25w/']

df_toComplete = read_any_path(spark, ",".join(file_paths), config)
df_toComplete_flat = df_toComplete.rdd.map(lambda x: json.loads(x.value)).toDF()
df_toComplete_flat

DataFrame[analysis: map<string,map<string,map<string,string>>>, channel: string, content: string, content_xml: string, ctime: bigint, gather: map<string,string>, mid: string, publisher: map<string,string>, title: string, url: string, user: map<string,bigint>, uuid: string, wtype: bigint]

In [24]:

file_paths = ['s3://private-cooperate-data/zh-web-baijiahao/Articles/baijiahao_Articles_202401.jsonl']
    # 's3://data-warehouse/sufutao/llm_anno/content_only/textbook_content_20pages_25w/']

df_toComplete = read_any_path(spark, ",".join(file_paths), config)
df_toComplete.printSchema()

root
 |-- value: string (nullable = true)
 |-- filename: string (nullable = true)



In [53]:
df_toComplete_flat = df_toComplete.rdd.map(lambda x: json.loads(x.value)).toDF()

In [7]:
    # create spark session
from pyspark.sql import Row, DataFrame

from app.common.spark_ext import *
from app.common.json_util import *

from app.common.s3 import *

config = {
    "spark_conf_name": "spark_2",
    "skip_success_check": True,
    # 根据个人路径进行替换
    "spark.executorEnv.LLM_WEB_KIT_CFG_PATH": "/share/jiangwenhao/.llm-web-kit.jsonc",
    "spark.dynamicAllocation.maxExecutors": "400",
}

# spark = new_spark_session("llm_kit_cc", config)
# spark

UsageError: Line magic function `%` not found.


In [25]:
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    FloatType,
    ArrayType,
    BooleanType
)

# 定义JSON模式
json_schema = StructType([
    # 顶层字段
    StructField("channel", StringType()),
    StructField("mid", StringType()),
    StructField("gather", StructType([
        StructField("site_domain", StringType()),
        StructField("stime", IntegerType()),
        StructField("info_flag_v5", StructType([
            StructField("common", ArrayType(StringType()))
        ])),
        StructField("site_name", StringType()),
        StructField("config_id", StringType()),
        StructField("gtime", IntegerType()),
        StructField("ICP", StringType()),
        StructField("data_type", StringType()),
        StructField("sub_domain", ArrayType(StringType())),
        StructField("stime_interval", FloatType()),
        StructField("interval", IntegerType()),
        StructField("info_flag", ArrayType(StringType()))
    ])),
    StructField("title", StringType()),
    StructField("analysis", StructType([
        StructField("info_src", StructType([
            StructField("loc", StructType([
                StructField("province", StringType()),
                StructField("city", StringType()),
                StructField("level", IntegerType()),
                StructField("name", ArrayType(StringType()))
            ])),
            StructField("domain", StringType()),
            StructField("name", StringType()),
            StructField("lv", StructType([
                StructField("level", StringType()),
                StructField("name", StringType())
            ])),
            StructField("cls", StructType([
                StructField("level", IntegerType()),
                StructField("name", ArrayType(StringType()))
            ])),
            StructField("type", IntegerType())
        ])),
        StructField("hashcode", StructType([
            StructField("5", IntegerType())
        ])),
        StructField("find_address", StructType([
            StructField("district_count", IntegerType()),
            StructField("province", ArrayType(StringType())),
            StructField("city", ArrayType(StringType())),
            StructField("province_count", IntegerType()),
            StructField("district", ArrayType(StringType())),
            StructField("words", ArrayType(StringType())),
            StructField("city_count", IntegerType())
        ]))
    ])),
    StructField("uuid", StringType()),
    StructField("url", StringType()),
    StructField("content", StringType()),
    StructField("wtype", IntegerType()),
    StructField("publisher", StructType([
        StructField("id", StringType()),
        StructField("platform", StringType())
    ])),
    StructField("ctime", IntegerType()),
    StructField("user", StructType([
        StructField("friends_count", IntegerType()),
        StructField("analysis_location", ArrayType(StringType())),
        StructField("like_count", IntegerType()),
        StructField("profile_img_url", StringType()),
        StructField("gender", StringType()),
        StructField("level", IntegerType()),
        StructField("uid_org", StringType()),
        StructField("verified", IntegerType()),
        StructField("description", StringType()),
        StructField("url", StringType()),
        StructField("uid", StringType()),
        StructField("statuses_count", IntegerType()),
        StructField("followers_count", IntegerType()),
        StructField("name", StringType()),
        StructField("nickname", StringType()),
        StructField("verified_reason", StringType())
    ])),
    StructField("content_xml", StringType())
])

In [26]:

parsed_df = df_toComplete.withColumn("parsed_value", from_json(df_toComplete["value"], json_schema))

In [50]:
from pyspark.sql.functions import explode
# 展开嵌套的字段
flattened_df = parsed_df.select(
    # 顶层字段
    "parsed_value.channel",
    "parsed_value.mid",
    "parsed_value.title",
    "parsed_value.uuid",
    "parsed_value.content",
    "parsed_value.ctime",
    "parsed_value.gather",
    "parsed_value.gather.site_name",
    "parsed_value.gather.config_id",
    "parsed_value.gather.gtime",
    "parsed_value.gather.ICP",
    "parsed_value.gather.data_type",
    "parsed_value.gather.sub_domain",
    "parsed_value.gather.stime_interval",
    "parsed_value.gather.interval",
    "parsed_value.gather.info_flag",
    "parsed_value.analysis",
    "parsed_value.url",
    "parsed_value.wtype",
    "parsed_value.publisher",
    "parsed_value.user.analysis_location",
    "parsed_value.user.like_count",
    "parsed_value.user.profile_img_url",
    "parsed_value.user.gender",
    "parsed_value.user.level",
    "parsed_value.user.uid_org",
    "parsed_value.user.verified",
    "parsed_value.user.description",
    "parsed_value.user.url",
    "parsed_value.user.uid",
    "parsed_value.user.statuses_count",
    "parsed_value.user.followers_count",
    "parsed_value.user.name",
    "parsed_value.user.nickname",
    "parsed_value.user.verified_reason",
    "parsed_value.content_xml"
)

In [51]:
flattened_df.show()

[Stage 4:>                                                          (0 + 1) / 1]

+-------+-------------------+--------------------------------------+--------------------+-------------------------------------+----------+--------------------+---------+---------+----------+---------------+---------+--------------------+--------------+--------+----------+--------+--------------------+-----+----------------------+----------------------------------+----------+--------------------+------+-----+----------------+--------+--------------------------------------+--------------------+--------------------+--------------+---------------+------------------+------------------+--------------------------+--------------------+
|channel|                mid|                                 title|                uuid|                              content|     ctime|              gather|site_name|config_id|     gtime|            ICP|data_type|          sub_domain|stime_interval|interval| info_flag|analysis|                 url|wtype|             publisher|                 analysis_locatio